# SVD - Descomposición en valores singulares

La SVD (Singular Value Decomposition) es una técnica del álgebra lineal que descompone cualquier matriz $A \in \mathbb{R}^{m \times n}$ en tres componentes:

$$A = U \Sigma V^T$$

donde:
- $U$: matriz de vectores singulares izquierdos (representa las direcciones principales de las filas).
- $\Sigma$: matriz diagonal de valores singulares ($\sigma_i$), que indican la “fuerza” o “importancia” de cada dimensión latente.
- $V$: matriz de vectores singulares derechos (direcciones principales de las columnas).

Estos valores singulares ordenan las dimensiones de mayor a menor importancia, permitiendo representar la estructura de los datos en un espacio reducido.

**Algunas aplicaciones:**

- Reducción de dimensionalidad: como base de PCA para eliminar redundancia en los datos.
- Compresión y denoising: reducción de ruido en imágenes o señales.
- Sistemas de recomendación: extracción de factores latentes usuario–ítem (FunkSVD, SVD++).
- Etc..

**Motivación de utiliza este algoritmo:**

La motivación de usar SVD es revelar la estructura esencial oculta en los datos.
Al descomponer una matriz en tres partes $A = U \Sigma V^$T, SVD:
- Separa la información (patrones importantes) del ruido.
- Reduce dimensionalidad, manteniendo solo las relaciones más fuertes.
- Permite representar datos complejos (como usuarios e ítems) en un espacio latente compacto y útil para predicción o interpretación.

## Aplicación en sistemas de recomendación

En sistemas de recomendación, la SVD (Descomposición en Valores Singulares) se utiliza para factorizar la matriz de interacciones usuario–ítem (por ejemplo, de ratings).
El objetivo es representar tanto a los usuarios como a los ítems en un espacio latente compartido, donde las relaciones se modelan a través de unos pocos factores latentes.

Existen dos enfoques principales adaptados a este contexto:
1.	SVD truncada:
En lugar de reconstruir la matriz original de manera exacta, se aproxima utilizando solo los k valores singulares más importantes.
Esto reduce la dimensionalidad y preserva las relaciones más relevantes entre usuarios e ítems.

2.	SVD ajustada sobre valores conocidos (FunkSVD, SVD++):
En este enfoque, no se aplica la SVD algebraica clásica, sino que se entrenan matrices latentes $U$ y $V$ (que absorben implícitamente a $\Sigma$) mediante un proceso iterativo que minimiza el error sobre los ratings observados.
Por ejemplo, si el usuario 1 calificó con 4 el ítem 3, y el usuario 4 calificó con 1 el ítem 5, el algoritmo ajusta los vectores correspondientes $U_1, V_3, U_4, V_5$ para que el producto punto $U_u \cdot V_i^T$ se aproxime lo más posible a las calificaciones reales.

$$R \approx U_k \Sigma_k V_k^T$$

donde k es el número de factores latentes.
Cada usuario e ítem se representa en el espacio latente como:

$$P = U_k \Sigma_k^{1/2}, \quad Q = V_k \Sigma_k^{1/2}$$

y la predicción de un rating se obtiene mediante:

$$\hat{r}_{ui} = P_u \cdot Q_i^T$$

### Importación de librerías

In [134]:
# !pip install cmfrec
# !pip install lightfm
# !pip install matrix-factorization

In [136]:
import pandas as pd
import numpy as np

In [137]:
from numpy.linalg import svd

### Lectura del conjunto de datos

En este ejemplo estaremos trabajando con el dataset [IMDB](https://ai.stanford.edu/~amaas/data/sentiment/)

In [ ]:
ruta_archivo = 'u.data'

In [139]:
df = pd.read_csv(ruta_archivo, sep='\t', names=['user_id','movie_id','rating','timestamp'])
df

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


### 1️⃣ SVD tradicional
- Definición: $A = U \Sigma V^T$
- Valores singulares ($\sigma_i$): importancia de cada componente / varianza explicada
- Uso: PCA, compresión, análisis espectral
- Requisito: matriz completa


In [140]:
# from numpy.linalg import svd

El metodo `svd` de NumPy espera una matriz de interacciones, por tanto construiremos una con el formato [user, item] y los ratings como valores

In [141]:
ratings_df = pd.pivot_table(data=df, values='rating', index='user_id', columns='movie_id')

Para acelerar los calculos, podemos guardar los valores de esta matriz de iteraaciones en formato array de NumPy:

In [ ]:
# ratings_df: DataFrame donde filas=usuarios, columnas=películas, valores=ratings
A = ratings_df.values  # forma (m, n), sin valores faltantes

In [ ]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)

LinAlgError: SVD did not converge

Como la mayoria de transformadores o estimadores, como `SVD` tradicional opera sobre todo el dataset y no solo sobre los valores conocidos, tenemos el problema de valores nulos.. Imputaremos por '0' para simplificar

In [144]:
# Reemplazar NaN por 0 (u otra estrategia, como la media del usuario)
A = np.nan_to_num(A, nan=0.0)  # reemplaza NaN por 0

In [ ]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
U.shape, s.shape, VT.shape

Vemos que $s$ tiene forma de vector unidimensional, ya que contiene únicamente los valores singulares.
Sin embargo, para poder realizar la reconstrucción matricial completa, necesitamos convertirlo en una matriz diagonal $Σ$, donde los elementos de $s$ se ubiquen en la diagonal principal.
Esta matriz debe tener tantas filas como $U$ y tantas columnas como $V^T$, de modo que las dimensiones sean compatibles en la multiplicación:

$A \approx U \, \Sigma \, V^T$

In [ ]:
Sigma = diag(s)
Sigma.shape

Podemos ver cuanto aporta cada vector singular (de la matriz izquierda y derecha: U y V_T) por medio de la matriz $\sigma$ que contiene los valores singulares asociados a cada uno:

In [ ]:
s

In [ ]:
sum(s)

In [ ]:
# Calcula el porcentaje de varianza explicada por cada valor singular
S / sum(S) * 100

In [ ]:
sum(S / sum(S) * 100)

Como vimos, la `SVD` puede emplearse para diversos fines (reducción de dimensionalidad, sistemas de recomendación, compresión de datos, etc.).
Una de sus aplicaciones más directas es la reconstrucción de la matriz original a partir de sus componentes.

Podemos optar por dos enfoques:
- Reconstrucción exacta, utilizando todos los vectores y valores singulares.
- Reconstrucción aproximada, empleando solo los k valores singulares más grandes, lo que reduce el costo computacional y, en muchos casos, elimina ruido asociado a dimensiones menos relevantes.

En este caso, al tratarse de una `SVD` tradicional (no truncada), realizaremos una reconstrucción exacta de la matriz original:

In [ ]:
# Reconstrucción exacta
A_hat = U @ Sigma @ VT
A_hat

array([[ 5.00000000e+00,  3.00000000e+00,  4.00000000e+00, ...,
        -2.55048397e-16, -1.56125113e-17,  5.68772460e-16],
       [ 4.00000000e+00,  3.43385043e-13, -8.13932255e-15, ...,
        -1.50920942e-16,  4.70110062e-16,  1.73472348e-17],
       [-2.69020917e-14,  1.23581700e-14,  1.43982049e-16, ...,
        -2.41993925e-16, -1.38777878e-16, -1.04733930e-16],
       ...,
       [ 5.00000000e+00,  9.57567359e-16,  5.83907922e-15, ...,
        -2.48932819e-16, -4.85722573e-16, -1.78676518e-16],
       [-4.15466272e-15,  8.39779635e-15, -1.70002901e-15, ...,
        -4.91794105e-16, -4.33680869e-17,  1.46150453e-16],
       [ 1.72188652e-14,  5.00000000e+00, -2.18575158e-15, ...,
        -3.50414142e-16,  2.87964097e-16, -2.65412692e-16]])

Al utilizar todos los vectores singulares de $U$ y $V_T$ y todos los valores sigulates de la matriz diagonal, la reconstrucción es exacta a la matriz original, no es una aproximación.

In [ ]:
A_hat[:10, :10], A[:10, :10]

### 2️⃣ SVD truncada para recomendación
- Definición: $A \approx U_k \Sigma_k V_k^T$
- Valores singulares: fuerza de factores latentes (patrones usuario–ítem)
- Uso: predicción de ratings, embeddings de usuarios e ítems ya que ahora estan en la mismo numero de dimensiones `k`
- Requisito: matriz completa o con estrategias de imputación para valores faltantes
- Ventaja: reduce dimensionalidad, captura patrones latentes
- Nota: no maneja sparsidad de manera nativa; requiere completitud o aproximación



In [148]:
ratings_df = pd.pivot_table(data=df, values='rating', index='user_id', columns='movie_id')

In [ ]:
# ratings_df: DataFrame donde filas=usuarios, columnas=películas, valores=ratings
A = ratings_df.values  # forma (m, n), sin valores faltantes

In [150]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
Sigma = np.diag(s)

LinAlgError: SVD did not converge

problema de valores nulos.. Imputaremos por '0' para simplificar

In [151]:
# Reemplazar NaN por 0 (u otra estrategia, como la media del usuario)
A = np.nan_to_num(A, nan=0.0)  # reemplaza NaN por 0

In [ ]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
U.shape, s.shape, VT.shape

Vemos que $s$ tiene forma de vector unidimensional, ya que contiene únicamente los valores singulares.
Sin embargo, para poder realizar la reconstrucción matricial completa, necesitamos convertirlo en una matriz diagonal $Σ$, donde los elementos de $s$ se ubiquen en la diagonal principal.
Esta matriz debe tener tantas filas como $U$ y tantas columnas como $V^T$, de modo que las dimensiones sean compatibles en la multiplicación:

$A \approx U \, \Sigma \, V^T$

In [ ]:
Sigma = diag(s)
Sigma.shape

Podemos ver cuanto aporta cada vector singular (de la matriz izquierda y derecha: U y V_T) por medio de la matriz $\sigma$ que contiene los valores singulares asociados a cada uno:

In [ ]:
s

In [ ]:
sum(s)

In [ ]:
# Calcula el porcentaje de varianza explicada por cada valor singular
S / sum(S) * 100

In [ ]:
sum(S / sum(S) * 100)

In [ ]:
k = 10

Como vimos, la `SVD` puede emplearse para diversos fines (reducción de dimensionalidad, sistemas de recomendación, compresión de datos, etc.).
Una de sus aplicaciones más directas es la reconstrucción de la matriz original a partir de sus componentes.

En este caso, al tratarse de una `SVD` trunkado, realizaremos una reconstrucción aproximada de la matriz original utilizando solo k componentes:

In [156]:
U_k = U[:, :k]
Sigma_k = np.diag(s[:k])
V_k = VT.T[:, :k]

In [157]:
# Reconstrucción aproximada
A_hat = U_k @ Sigma_k @ V_k.T
A_hat

array([[ 4.01688618e+00,  2.10514989e+00,  1.37439578e+00, ...,
        -3.83174025e-03,  2.64379479e-02,  7.33806834e-02],
       [ 1.99295186e+00, -1.81693234e-02, -4.30251308e-03, ...,
         9.28078660e-03, -2.66167393e-03, -2.95256660e-02],
       [-1.74774668e-01, -5.60278496e-02,  1.81311614e-01, ...,
         2.16443047e-02,  1.91721169e-03,  2.49374917e-04],
       ...,
       [ 2.23624102e+00,  3.89103327e-02,  2.76077652e-01, ...,
        -5.53764648e-03,  1.07306047e-02, -4.23773725e-03],
       [ 1.31024409e+00,  1.55156267e-01, -4.75132794e-01, ...,
         1.50293142e-02,  1.15206267e-02, -2.94522888e-02],
       [ 1.72204325e+00,  2.05335046e+00,  1.18288029e+00, ...,
        -9.89971558e-03,  1.73695571e-02,  2.05937102e-02]])

Cuando seleccionamos los 'k' valores y vectores singulares, solo podremos aproximar la solción pero con mucho menor costo computacional asociado.

In [ ]:
A_hat, A

Como vimos, ahora tenemos a usuarios y a itmes con `k` factores latentes, es decir que podemos hubicarlos en un mismo hiperplano y buscar similaridades en ese espacio geometrico e incluso si k = 2 o 3, se podria graficar para ver dichas relaciones:

In [158]:
# Crear embeddings de usuarios e ítems
X_users = U_k @ np.sqrt(Sigma_k)
Y_items = V_k @ np.sqrt(Sigma_k)

### 3️⃣ SVD iterativa (FunkSVD, SVD++)
- Definición: optimización iterativa
$\min_{P,Q} \sum_{(u,i) \in \Omega} (r_{ui} - p_u^T q_i)^2 + \lambda(\|p_u\|^2 + \|q_i\|^2)$
- Valores singulares: implícitos; $P$ y $Q$ se ajustan por gradiente o ALS
- Uso: recomendadores modernos, escalables, con regularización e interacciones implícitas
- Requisito: no necesita matriz completa; solo requiere los ratings observados
- Ventaja: funciona directamente con matrices muy dispersas y generaliza patrones latentes

Implementación paso a paso:


In [ ]:
# Mapear IDs a índices consecutivos
user_mapping = {id_: idx for idx, id_ in enumerate(df['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(df['movie_id'].unique())}

In [ ]:
df['user_idx'] = df['user_id'].map(user_mapping)
df['item_idx'] = df['movie_id'].map(item_mapping)

In [160]:
n_users = df['user_idx'].nunique()
n_items = df['item_idx'].nunique()
k = 20

In [161]:
# Inicialización de factores y biases
P = np.random.normal(scale=0.1, size=(n_users, k))
Q = np.random.normal(scale=0.1, size=(n_items, k))
bu = np.zeros(n_users)  # bias usuario
bi = np.zeros(n_items)  # bias ítem
global_mean = df['rating'].mean()

In [162]:
lr = 0.005
lambda_reg = 0.02
n_epochs = 10

In [163]:
rows = df['user_idx'].values
cols = df['item_idx'].values
vals = df['rating'].values

In [164]:
# Entrenamiento FunkSVD con SGD
for epoch in range(n_epochs):
    for u, i, r_ui in zip(rows, cols, vals):
        pred = global_mean + bu[u] + bi[i] + P[u] @ Q[i].T
        err = r_ui - pred
        # Actualizar factores y biases
        bu[u] += lr * (err - lambda_reg * bu[u])
        bi[i] += lr * (err - lambda_reg * bi[i])
        P[u] += lr * (err * Q[i] - lambda_reg * P[u])
        Q[i] += lr * (err * P[u] - lambda_reg * Q[i])
    print(f"Epoch {epoch+1} finalizada")

Epoch 1 finalizada
Epoch 2 finalizada
Epoch 3 finalizada
Epoch 4 finalizada
Epoch 5 finalizada
Epoch 6 finalizada
Epoch 7 finalizada
Epoch 8 finalizada
Epoch 9 finalizada
Epoch 10 finalizada


In [165]:
ratings_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
df.head()

,user_id,movie_id,rating,timestamp,user_idx,item_idx
0,196,242,3,881250949,0,0
1,186,302,3,891717742,1,1
2,22,377,1,878887116,2,2
3,244,51,2,880606923,3,3
4,166,346,1,886397596,4,4


In [169]:
print("Usuarios en mapping:", list(user_mapping.keys())[:10])
print("Películas en mapping:", list(item_mapping.keys())[:10])

Usuarios en mapping: [np.int64(196), np.int64(186), np.int64(22), np.int64(244), np.int64(166), np.int64(298), np.int64(115), np.int64(253), np.int64(305), np.int64(6)]
Películas en mapping: [np.int64(242), np.int64(302), np.int64(377), np.int64(51), np.int64(346), np.int64(474), np.int64(265), np.int64(465), np.int64(451), np.int64(86)]


In [167]:
# Predicción ejemplo
user_id = df['user_id'].iloc[4]
item_id = df['movie_id'].iloc[3]
u_idx = user_mapping[user_id]
i_idx = item_mapping[item_id]

In [168]:
rating_pred = global_mean + bu[u_idx] + bi[i_idx] + P[u_idx] @ Q[i_idx].T
rating_pred

np.float64(3.621485302232855)